In [88]:
%reload_ext autoreload
%autoreload 2

import os
import openai
import numpy as np
import utils

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [89]:
hellaswag = utils.load_raw_dataset("hellaswag", None)
hellaswag[0]

[2023-01-16 04:54:32,445] [datasets.builder] [builder.py:785] Found cached dataset hellaswag (/workspaces/seed/cache/hf_dataset/hellaswag/default/0.1.0/c37cd37196278995f42bc32f532730ae9b0d5f0f4a2d3b97735c17ff3ad67169)


{'ind': 24,
 'activity_label': 'Roof shingle removal',
 'ctx_a': 'A man is sitting on a roof.',
 'ctx_b': 'he',
 'ctx': 'A man is sitting on a roof. he',
 'endings': ['is using wrap to wrap a pair of skis.',
  'is ripping level tiles off.',
  "is holding a rubik's cube.",
  'starts pulling up roofing on a roof.'],
 'source_id': 'activitynet~v_-JhWjGDPHMY',
 'split': 'val',
 'split_type': 'indomain',
 'label': '3'}

In [90]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [117]:
def seq2logprob(text):
    res = openai.Completion.create(
        model='text-curie-001',
        prompt=text,
        max_tokens=0,
        echo=True,
        temperature=0,
        logprobs=1,
    )
    
    seq_logprob = []

    for i in range(len(text)):
        choice = res.choices[i]
        logprobs = choice.logprobs.token_logprobs
        logprobs = [p for p in logprobs if p is not None]
        seq_logprob.append(sum(logprobs))

    return np.array(seq_logprob)

In [119]:
FOR_REAL = False
n = 100
accurate = 0
total_tokens = 0

for inst in hellaswag.select(range(n)):
    target = int(inst['label'])

    candidates = [inst['ctx'] + ' ' + option for option in inst['endings']]

    n_tokens = [len(tokenizer.encode(c)) for c in candidates]
    total_tokens += sum(n_tokens)

    if FOR_REAL:
        try:
            seq_logprob = seq2logprob(candidates)
            pred = seq_logprob.argmax()

            if pred == target:
                accurate += 1
        except:
            print(f"API error on {inst}")
            n-=1
            continue
        

print(f"accuracy= {accurate / n} with {n} examples")
print(f"The cost of this run for Curies is {total_tokens * 1e-3 * 2e-3:.2f} USD. Davinci is 10x.")


API error on {'ind': 386, 'activity_label': 'Washing face', 'ctx_a': 'A black female is shown in a room with a black scarf around her head. Black spots on her faced are then zoomed in on and she begins to apply something on it.', 'ctx_b': 'next', 'ctx': 'A black female is shown in a room with a black scarf around her head. Black spots on her faced are then zoomed in on and she begins to apply something on it. next', 'endings': [', she begins to brush her lips and make painted lips look just like a lipstick to the camera and ends with her closing gift and instruction showing her off at the end.', 'she is shown in various different locations and gets extremely close to a wall.', ', she is shown making multiple moans and sprays some kind of liquid down her face before hanging up.', ', she takes some spray and it makes her skin extremely oily and she continues to talk until the video is over.'], 'source_id': 'activitynet~v_33SI8z8PovA', 'split': 'val', 'split_type': 'zeroshot', 'label': '3

`Curie` without prompting: acc = 0.48  
`Davinci` without prompting: acc = 0.53

I don't see Davinci result as model failure. This raises questions about proper interpretation of eval result. Approximate multiple choice with `seequence logprob` is not a fair eval especially for sentence completion task. Sentence completion is not like multiple choice quiz on physics that has easy right answer. 